In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [5]:
ab_data = pd.read_csv('../dataset/ab_data.csv')
ab_data.head()
#음 대강 보니까 길이와 무게로 종을 예측하는 것이네요!

,Weight,Length,Species
0,242,25.4,A
1,290,26.3,A
2,340,26.5,A
3,363,29.0,A
4,430,29.0,A


In [13]:
#결측치 확인
ab_data.info()
#겉보기엔 이상치가 없어보임
print(ab_data.describe())
#그러나? 0도 결측치이자 될 수 있음. 그리고 범위가 너무 큼


for i in ab_data.columns:
    print(ab_data[ab_data[i] == 0])
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Weight   55 non-null     int64  
 1   Length   55 non-null     float64
 2   Species  55 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ KB
            Weight     Length
count    55.000000  55.000000
mean    448.454545  29.169091
std     285.422936   6.479965
min       0.000000  14.100000
25%     165.000000  23.450000
50%     450.000000  30.000000
75%     682.500000  34.000000
max    1000.000000  41.000000
    Weight  Length Species
40       0    20.5       B
Empty DataFrame
Columns: [Weight, Length, Species]
Index: []
Empty DataFrame
Columns: [Weight, Length, Species]
Index: []


In [34]:
#길이가 비슷한 종 B의 값을 대체하는 건 어떔?
#   Weight  Length Species
#   0        20.5       B

for_mean_Weight_df = ab_data[(ab_data.Length< 25)& (ab_data.Length>=20)]
for_mean_Weight_df = for_mean_Weight_df[for_mean_Weight_df.Weight != 0]
mean_weight= for_mean_Weight_df['Weight'].mean()
ab_data.replace(0,mean_weight, inplace=True)
target_data = ab_data['Species']
feature_data = ab_data.drop(['Species'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.2, random_state =12)
knn = KNeighborsClassifier()
param = {'n_neighbors': [i for i in range(1,55)], 'weights': ['uniform', 'distance'],'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],'p':[1,2,3,4]}
Gridknn = GridSearchCV(knn, param_grid= param, cv=5,refit=True)
Gridknn.fit(X_train, y_train)

c:\Users\dddf\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\dddf\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\dddf\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 430, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\Users\dddf\anaconda3\lib\site-packages\sklearn\base.py", line 638, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "c:\Users\dddf\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 234, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "c:\Users\dddf\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 810, in kneighbors
    raise Val

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'p': [1, 2, 3, 4],
                         'weights': ['uniform', 'distance']})

In [44]:
print(f'best_params = {Gridknn.best_params_}')
print(f'best_score = {round(Gridknn.best_score_,2)}')
print(f'best_estimator = {Gridknn.best_estimator_}')
best_knn_est = Gridknn.best_estimator_
y_pred= best_knn_est.predict(X_test)
print(f'accuary_score = {round(accuracy_score(y_test,y_pred),2)}')

best_params = {'algorithm': 'auto', 'n_neighbors': 2, 'p': 1, 'weights': 'uniform'}
best_score = 0.95
best_estimator = KNeighborsClassifier(n_neighbors=2, p=1)
accuary_score = 0.91
